In [ ]:
from glob import glob

dirs = glob("./data/v1/*") + glob("./data/v1/*")
dirs

In [ ]:
from glob import glob
import numpy as np
from utils import read_yaml
from pprint import pprint


paths = glob("./training_results/*/*/*test_debug*")
results = {
    "v1": {2: [], 4: [], 8: [], 16: [], 32: [], 64: []},
    "v1_pretrained": {2: [], 4: [], 8: [], 16: [], 32: [], 64: []},
    "v2": {2: [], 4: [], 8: [], 16: [], 32: [], 64: []},
    "v2_pretrained": {2: [], 4: [], 8: [], 16: [], 32: [], 64: []},
}
for path in paths:
    if "/v1/" in path:
        version = "v1"
    elif "/v2/" in path:
        version = "v2"
    else:
        raise ValueError

    mean_ = float(path.split("test_debug-mean=")[1].split("-std")[0])
    capacity = int(path.split("_capacity=")[1].split("_seed=")[0])
    pretrained = path.split("pretrain_semantic=")[1].split("_varying_rewards")[0]
    if pretrained == "False":
        pretrained = False
    elif pretrained == "True":
        pretrained = True
    else:
        raise ValueError

    if pretrained:
        version += "_pretrained"

    results[version][capacity].append(mean_)

for key, val in results.items():
    for key_, val_ in val.items():
        assert len(val_) == 5

results = {
    key: {
        key_: {
            "mean": round(np.mean(val_).item(), 4),
            "std": round(np.std(val_).item(), 4),
        }
        for key_, val_ in val.items()
    }
    for key, val in results.items()
}
pprint(results)

In [ ]:
from glob import glob
import shutil
import os
from utils import read_yaml

old_dirs = []
new_dirs = []
for foo in glob("./training_results/*"):
    bar = glob(os.path.join(foo, "*/*/*test*"))
    if len(bar) == 0:
        continue
    bar = bar[0]
    old_dir = "/".join(bar.split("/")[:-1])
    old_dirs.append(old_dir)
    hparams = read_yaml(os.path.join(old_dir, "hparams.yaml"))

    des_version = hparams["des_version"]
    allow_random_human = hparams["allow_random_human"]
    allow_random_question = hparams["allow_random_question"]
    pretrain_semantic = hparams["pretrain_semantic"]
    varying_rewards = hparams["varying_rewards"]
    capacity = hparams["capacity"]["episodic"] + hparams["capacity"]["semantic"]
    des_size = hparams["des_size"]
    seed = hparams["seed"]

    new_dir = (
        f"training_results/"
        f"des_version={des_version}_"
        f"allow_random_human={allow_random_human}_"
        f"allow_random_question={allow_random_question}_"
        f"pretrain_semantic={pretrain_semantic}_"
        f"varying_rewards={varying_rewards}_"
        f"des_size={des_size}_"
        f"capacity={capacity}_"
        f"seed={seed}"
    )
    new_dirs.append(new_dir)
    os.rename(old_dir, new_dir)

In [ ]:
import logging

logger = logging.getLogger()
logger.disabled = True

from room_env.utils import print_handcrafted


print_handcrafted(
    env="RoomEnv-v2",
    des_size="l",
    seeds=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    question_prob=0.1,
    policies={
        "memory_management": "rl",
        "question_answer": "episodic_semantic",
        "encoding": "argmax",
    },
    capacities=[2, 4, 8, 16, 32, 64],
    allow_random_human=False,
    allow_random_question=False,
    varying_rewards=False,
    check_resources=True,
    version="v2",
)

In [ ]:
import logging

logger = logging.getLogger()
logger.disabled = True

from room_env.utils import print_handcrafted


print_handcrafted(
    env="RoomEnv-v2",
    des_size="l",
    seeds=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    question_prob=1.0,
    policies={
        "memory_management": "rl",
        "question_answer": "episodic_semantic",
        "encoding": "argmax",
    },
    capacities=[2, 4, 8, 16, 32, 64],
    allow_random_human=False,
    allow_random_question=False,
    varying_rewards=False,
    check_resources=True,
    version="v1",
)

In [ ]:
from glob import glob
import numpy as np

paths = glob("./training_results_AMD/*/test_debug*")
categories = sorted(
    list(set([path.split("/")[-2].split("-seed")[0] for path in paths]))
)
categories = [foo for foo in categories if "-gamma=0.99-" not in foo]

foo = [
    (
        category,
        np.mean(
            [
                float(path.split("/")[-1].split("-mean=")[-1].split("-std=")[0])
                for path in paths
                if category in path
            ]
        )
        .round(3)
        .item(),
    )
    for category in categories
]
foo

In [ ]:
import logging

logger = logging.getLogger()
logger.disabled = True

from glob import glob
import os
from utils import read_yaml

# des_size=l-capacity=2-pretrain=True-gpus=0-seed=0

for path in glob("./training_results_AMD/*/*/version*/test_debug-*"):
    dir_name_old = "/".join(path.split("/")[:-1])
    hparams_path = os.path.join(dir_name_old, "hparams.yaml")
    hparams = read_yaml(hparams_path)
    des_size = hparams["des_size"]
    capacity = hparams["capacity"]
    capacity = capacity["episodic"] + capacity["semantic"]
    allow_random_human = hparams["allow_random_human"]
    allow_random_question = hparams["allow_random_question"]
    pretrain_semantic = hparams["pretrain_semantic"]
    varying_rewards = hparams["varying_rewards"]
    gamma = hparams["gamma"]
    gpus = hparams["gpus"]
    seed = hparams["seed"]
    dir_name_new = (
        f"./training_results_AMD/des_size={des_size}-capacity={capacity}-"
        f"allow_random_human={allow_random_human}-allow_random_question={allow_random_question}-"
        f"pretrain={pretrain_semantic}-varying_rewards={varying_rewards}-gamma={gamma}-"
        f"gpus={gpus}-seed={seed}"
    )
    print(dir_name_old, dir_name_new)
    os.rename(dir_name_old, dir_name_new)

In [ ]:
import logging

logger = logging.getLogger()
logger.disabled = True
import room_env
from room_env import utils

In [ ]:
for allow_random_human in [True, False]:
    for allow_random_question in [True, False]:

        print(
            f"allow_random_human: {allow_random_human}, allow_random_question: {allow_random_question}"
        )

        utils.print_handcrafted(
            env="RoomEnv-v2",
            des_size="l",
            seeds=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
            question_prob=0.1,
            policies={
                "memory_management": "rl",
                "question_answer": "episodic_semantic",
                "encoding": "argmax",
            },
            capacities=[32],
            allow_random_human=allow_random_human,
            allow_random_question=allow_random_question,
            varying_rewards=False,
        )

In [ ]:
from copy import deepcopy
from glob import glob
from utils import read_yaml, read_json, write_yaml

foo = read_yaml("./train.yaml")

for capacity in [2, 4, 8, 16, 32, 64]:
    for pretrain_semantic in [True, False]:
        for seed in [0, 1, 2, 3, 4]:
            foo = deepcopy(foo)
            foo["pretrain_semantic"] = pretrain_semantic
            foo["capacity"] = {
                "episodic": capacity // 2,
                "semantic": capacity // 2,
                "short": 1,
            }
            foo["seed"] = seed
            write_yaml(foo, f"{capacity}_{pretrain_semantic}_{seed}.yaml")

In [ ]:
from glob import glob
import os
import numpy as np
from utils import read_yaml, read_json

results_ = []
for path in glob("./training_results_AMD/*/*/test_debug-*"):
    dir_name_old = "/".join(path.split("/")[:-1])
    hparams_path = os.path.join(dir_name_old, "hparams.yaml")
    hparams = read_yaml(hparams_path)
    des_size = hparams["des_size"]
    capacity = hparams["capacity"]
    capacity = capacity["episodic"] + capacity["semantic"]
    pretrain_semantic = hparams["pretrain_semantic"]
    gpus = hparams["gpus"]
    seed = hparams["seed"]

    test_mean = float(path.split("test_debug-mean=")[-1].split("-std")[0])
    results_.append((des_size, capacity, pretrain_semantic, seed, test_mean))

results = {}
for capacity in [2, 4, 8, 16, 32, 64]:
    for pretrain_semantic in [False, True]:
        results[f"capacity={capacity}_pretrain_semantic={pretrain_semantic}"] = round(
            np.mean(
                [
                    foo[-1]
                    for foo in results_
                    if foo[1] == capacity and foo[2] == pretrain_semantic
                ]
            ),
            3,
        ).item(), round(
            np.std(
                [
                    foo[-1]
                    for foo in results_
                    if foo[1] == capacity and foo[2] == pretrain_semantic
                ]
            ).item(),
            3,
        )
results

In [ ]:
import logging

logger = logging.getLogger()
logger.disabled = True

from room_env.utils import print_handcrafted

print_handcrafted(
    env="RoomEnv-v2",
    des_size="l",
    seeds=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    question_prob=0.1,
    policies={
        "memory_management": "rl",
        "question_answer": "episodic_semantic",
        "encoding": "argmax",
    },
    capacities=[2, 4, 8, 16, 32, 64],
)

In [ ]:
path

In [ ]:
hparams["allow_random_human"]

In [ ]:
import gym
import room_env

capacity = 32
env = gym.make(
    "RoomEnv-v2",
    des_size="l",
    seed=0,
    policies={
        "memory_management": "rl",
        "question_answer": "episodic_semantic",
        "encoding": "argmax",
    },
    capacity={"episodic": capacity // 2, "semantic": capacity // 2},
    question_prob=0.1,
    observation_params="perfect",
    allow_random_human=True,
    allow_random_question=True,
    pretrain_semantic=False,
    check_resources=False,
)
state, info = env.reset()

# Plotting


## Average sum of rewards per episode, validation (5 random runs)


In [ ]:
from plotting import plot_training_validation_results

plot_training_validation_results(
    data_dir="./data/",
    kind="val_total_reward_mean",
    capacity=32,
    save_dir="./figures/",
    ymin=64,
    ymax=128,
    ylog=False,
    xlabel="Step",
    figsize=(10, 10),
    legend_loc="lower right",
)

## Average training loss (5 random runs)


In [ ]:
plot_training_validation_results(
    data_dir="./data/",
    kind="train_loss",
    capacity=32,
    save_dir="./figures/",
    ymin=None,
    ymax=None,
    ylog=True,
    xlabel="Step",
    figsize=(10, 10),
    legend_loc="upper right",
)

## Average sum of rewards per episode, test (5 random runs), capacity=32


In [ ]:
from plotting import plot_test_results

plot_test_results()

## Average sum of rewards per episode, test (5 random runs), capacity=32, all sizes


In [ ]:
from plotting import plot_test_results_all_capacities
import matplotlib.pyplot as plt

plot_test_results_all_capacities()

## Load the q values


In [ ]:
from utils import read_json

test_debug_scratch_path = "./models/des_size=l-capacity=32-pretrain=False-gpus=0-seed=1/test_debug-mean=115.0-std=3.225.json"

test_debug_scratch = read_json(test_debug_scratch_path)["0"]

test_debug_pretrained_path = "./models/des_size=l-capacity=32-pretrain=True-gpus=0-seed=2/test_debug-mean=117.5-std=2.54.json"
test_debug_pretrained = read_json(test_debug_pretrained_path)["0"]

for step in [86]:
    print(f"STEP: {step}")
    print()
    print(f"SCRATCH: {test_debug_scratch[step]}")
    print()
    print(f"PRETRAINED: {test_debug_pretrained[step]}")
    print()
    print(
        "----------------------------------------------------------------------------"
    )

## Load the trained models and visualize the t-SNE-reduced embeddings.


In [ ]:
from plotting import UnderstandModel

um = UnderstandModel()

um.compute_reduction(
    tsne_params={
        "n_components": 2,
        "perplexity": 5,
        "init": "pca",
        "learning_rate": "auto",
    }
)

print(
    len(um.indices["humans"]),
    len(um.indices["objects"]),
    len(um.indices["object_locations"]),
)

for semantic in ["scratch", "pretrained"]:
    for interest in ["humans", "objects", "object_locations"]:
        um.plot_embeddings(semantic=semantic, interest=interest)